<a href="https://colab.research.google.com/github/yifeiw16/BigQuery-Geotab-Intersection-Congestion/blob/main/Stage1-Baseline/BigQuery_Traffic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
train_csv_path = "/content/drive/My Drive/CS412/train.csv"
#train_csv_path = '/kaggle/input/bigquery-geotab-intersection-congestion/train.csv'
df = pd.read_csv(train_csv_path)
print(df.head())

Mounted at /content/drive
     RowId  IntersectionId   Latitude  Longitude  \
0  1921357               0  33.791659 -84.430032   
1  1921358               0  33.791659 -84.430032   
2  1921359               0  33.791659 -84.430032   
3  1921360               0  33.791659 -84.430032   
4  1921361               0  33.791659 -84.430032   

                EntryStreetName                ExitStreetName EntryHeading  \
0  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   
1  Marietta Boulevard Northwest  Marietta Boulevard Northwest           SE   
2  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   
3  Marietta Boulevard Northwest  Marietta Boulevard Northwest           SE   
4  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   

  ExitHeading  Hour  Weekend  ...  TimeFromFirstStop_p40  \
0          NW     0        0  ...                    0.0   
1          SE     0        0  ...                    0.0   
2          N

In [ ]:
df = pd.read_csv(train_csv_path)
x_train = df[['IntersectionId','EntryHeading','ExitHeading']]
x_train = pd.concat([x_train,pd.get_dummies(df["City"],dummy_na=False, drop_first=False, dtype=int)],axis=1)
print(x_train.head())
heading_map = {'N':1,'NE':2,'E':3,'SE':4,'S':5, 'SW':6, 'W':7, 'NW': 8}
x_train['EntryHeading'] = x_train['EntryHeading'].replace(heading_map)
x_train['ExitHeading'] = x_train['ExitHeading'].replace(heading_map)
x_train['EntryHeading'] = x_train['EntryHeading'].replace(heading_map)
x_train['ExitHeading'] = x_train['ExitHeading'].replace(heading_map)
print(x_train)


   IntersectionId EntryHeading ExitHeading  Atlanta  Boston  Chicago  \
0               0           NW          NW        1       0        0   
1               0           SE          SE        1       0        0   
2               0           NW          NW        1       0        0   
3               0           SE          SE        1       0        0   
4               0           NW          NW        1       0        0   

   Philadelphia  
0             0  
1             0  
2             0  
3             0  
4             0  


/tmp/ipython-input-1820002226.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x_train['EntryHeading'] = x_train['EntryHeading'].replace(heading_map)


        IntersectionId  EntryHeading  ExitHeading  Atlanta  Boston  Chicago  \
0                    0             8            8        1       0        0   
1                    0             4            4        1       0        0   
2                    0             8            8        1       0        0   
3                    0             4            4        1       0        0   
4                    0             8            8        1       0        0   
...                ...           ...          ...      ...     ...      ...   
856382            1990             5            7        0       0        0   
856383            1990             5            5        0       0        0   
856384            1990             5            5        0       0        0   
856385            1990             7            7        0       0        0   
856386            1990             5            5        0       0        0   

        Philadelphia  
0                  0  
1    

/tmp/ipython-input-1820002226.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x_train['ExitHeading'] = x_train['ExitHeading'].replace(heading_map)


In [ ]:
# Hour: 0-23 cyclical -> [sin(2pi*hour/24), cos(2pi*hour/24)]
df['hour_sin'] = np.sin(2*np.pi*df['Hour']/24)
df['hour_cos'] = np.cos(2*np.pi*df['Hour']/24)
print(df.head())
print(df['Month'].unique())

df['month_sin'] = np.sin(2*np.pi*df['Month']/12)
df['month_cos'] = np.cos(2*np.pi*df['Month']/12)
print(df.head())

     RowId  IntersectionId   Latitude  Longitude  \
0  1921357               0  33.791659 -84.430032   
1  1921358               0  33.791659 -84.430032   
2  1921359               0  33.791659 -84.430032   
3  1921360               0  33.791659 -84.430032   
4  1921361               0  33.791659 -84.430032   

                EntryStreetName                ExitStreetName EntryHeading  \
0  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   
1  Marietta Boulevard Northwest  Marietta Boulevard Northwest           SE   
2  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   
3  Marietta Boulevard Northwest  Marietta Boulevard Northwest           SE   
4  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   

  ExitHeading  Hour  Weekend  ...  TimeFromFirstStop_p60  \
0          NW     0        0  ...                    0.0   
1          SE     0        0  ...                    0.0   
2          NW     1        0  ...     

In [ ]:
x_train['hour_sin'] = df['hour_sin']
x_train['hour_cos'] = df['hour_cos']
x_train['Hour'] = df['Hour']
# x_train['month_sin'] = df['month_sin']
# x_train['month_cos'] = df['month_cos']
x_train['Month'] = df['Month']
x_train['Weekend'] = df['Weekend']
print(x_train)

        IntersectionId  EntryHeading  ExitHeading  Atlanta  Boston  Chicago  \
0                    0             8            8        1       0        0   
1                    0             4            4        1       0        0   
2                    0             8            8        1       0        0   
3                    0             4            4        1       0        0   
4                    0             8            8        1       0        0   
...                ...           ...          ...      ...     ...      ...   
856382            1990             5            7        0       0        0   
856383            1990             5            5        0       0        0   
856384            1990             5            5        0       0        0   
856385            1990             7            7        0       0        0   
856386            1990             5            5        0       0        0   

        Philadelphia  hour_sin      hour_cos  Hour 

In [ ]:
y_train = pd.DataFrame()
y_train['TotalTimeStopped_p20'] = df['TotalTimeStopped_p20']
y_train['TotalTimeStopped_p50'] = df['TotalTimeStopped_p50']
y_train['TotalTimeStopped_p80'] = df['TotalTimeStopped_p80']
y_train['DistanceToFirstStop_p20'] = df['DistanceToFirstStop_p20']
y_train['DistanceToFirstStop_p50'] = df['DistanceToFirstStop_p50']
y_train['DistanceToFirstStop_p80'] = df['DistanceToFirstStop_p80']
print(y_train.head())

   TotalTimeStopped_p20  TotalTimeStopped_p50  TotalTimeStopped_p80  \
0                   0.0                   0.0                   0.0   
1                   0.0                   0.0                   0.0   
2                   0.0                   0.0                   0.0   
3                   0.0                   0.0                   0.0   
4                   0.0                   0.0                   0.0   

   DistanceToFirstStop_p20  DistanceToFirstStop_p50  DistanceToFirstStop_p80  
0                      0.0                      0.0                      0.0  
1                      0.0                      0.0                      0.0  
2                      0.0                      0.0                      0.0  
3                      0.0                      0.0                      0.0  
4                      0.0                      0.0                      0.0  


In [ ]:
#Split data into train and val
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

Baseline-Linear Regression

In [ ]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
reg = LinearRegression().fit(x_train, y_train)
y_pred = reg.predict(x_val)
print(y_pred.shape)
rmse_time_p20 = np.sqrt(mean_squared_error(y_val['TotalTimeStopped_p20'], y_pred[:,0]))
rmse_time_p50 = np.sqrt(mean_squared_error(y_val['TotalTimeStopped_p50'], y_pred[:,1]))
rmse_time_p80 = np.sqrt(mean_squared_error(y_val['TotalTimeStopped_p80'], y_pred[:,2]))
rmse_distance_p20 = np.sqrt(mean_squared_error(y_val['DistanceToFirstStop_p20'], y_pred[:,3]))
rmse_distance_p50 = np.sqrt(mean_squared_error(y_val['DistanceToFirstStop_p50'], y_pred[:,4]))
rmse_distance_p80 = np.sqrt(mean_squared_error(y_val['DistanceToFirstStop_p80'], y_pred[:,5]))
print("TotalTimeStopped_p20 RMSE:", rmse_time_p20)
print("TotalTimeStopped_p50 RMSE:", rmse_time_p50)
print("TotalTimeStopped_p80 RMSE:", rmse_time_p80)
print("DistanceToFirstStop_p20 RMSE:", rmse_distance_p20)
print("DistanceToFirstStop_p50 RMSE:", rmse_distance_p50)
print("DistanceToFirstStop_p80 RMSE:", rmse_distance_p80)
rmse_total = np.sqrt(mean_squared_error(y_val, y_pred))
print("Total RMSE:", rmse_total)

(171278, 6)
TotalTimeStopped_p20 RMSE: 7.115312016397103
TotalTimeStopped_p50 RMSE: 15.442253865293726
TotalTimeStopped_p80 RMSE: 27.463247478383668
DistanceToFirstStop_p20 RMSE: 31.203476166239607
DistanceToFirstStop_p50 RMSE: 76.0300686620512
DistanceToFirstStop_p80 RMSE: 159.7359035250612
Total RMSE: 74.51298317762135


Baseline-XGBoost

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor

scaler_y = StandardScaler()

y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled = scaler_y.transform(y_val)

# confirm
print("y_train_scaled mean (should be ~0):", y_train_scaled.mean(axis=0))
print("y_train_scaled std (should be ~1):", y_train_scaled.std(axis=0))

base_model = xgb.XGBRegressor(
    objective='reg:squarederror', # regression
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=10,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method='hist',
    eval_metric='rmse',
    random_state=42,
    device='cuda'
)

model = MultiOutputRegressor(base_model)

model.fit(x_train, y_train_scaled)

y_pred_scaled = model.predict(x_val)

y_pred = scaler_y.inverse_transform(y_pred_scaled)

rmse = np.sqrt(mean_squared_error(y_val,y_pred))
print('validation RMSE:',rmse)

rmses = np.sqrt(np.mean((y_val - y_pred)**2, axis=0))
print("Per-target RMSE:", rmses)



y_train_scaled mean (should be ~0): [ 3.58741065e-17 -8.05845064e-18  4.51356206e-17  8.48263420e-17
  6.01531708e-18 -1.78302295e-16]
y_train_scaled std (should be ~1): [1. 1. 1. 1. 1. 1.]


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [19:21:25] WARNING: /workspace/src/context.cc:53: No visible GPU is found, setting device to CPU.
  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [19:21:25] WARNING: /workspace/src/context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [19:22:27] WARNING: /workspace/src/context.cc:53: No visible GPU is found, setting device to CPU.
  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [19:22:27] WARNING: /workspace/src/context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/

validation RMSE: 57.755715355711764
Per-target RMSE: TotalTimeStopped_p20         6.197466
TotalTimeStopped_p50        12.067221
TotalTimeStopped_p80        20.963552
DistanceToFirstStop_p20     28.738361
DistanceToFirstStop_p50     62.791642
DistanceToFirstStop_p80    120.922111
dtype: float64


In [ ]:
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

#
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train_log)
y_val_scaled = scaler_y.transform(y_val_log)

#
pred_val_scaled = np.zeros_like(y_val_scaled)

for i, target in enumerate(y_train.columns):
    print(f"Training target: {target}")

    dtrain = xgb.DMatrix(x_train, label=y_train_scaled[:, i])
    dval = xgb.DMatrix(x_val, label=y_val_scaled[:, i])

    params = {
        'objective': 'reg:squarederror',
        'tree_method': 'hist',
        'device': 'cuda',
        'eval_metric': 'rmse',
        'max_depth': 10,
        'eta': 0.01,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'random_state': 42
    }

    watchlist = [(dtrain, 'train'), (dval, 'eval')]
    model = xgb.train(params, dtrain, num_boost_round=6000, evals=watchlist,
                      early_stopping_rounds=50, verbose_eval=50)

    # predict
    pred_val_scaled[:, i] = model.predict(dval)

# inverse
pred_val_log = scaler_y.inverse_transform(pred_val_scaled)
y_pred = np.expm1(pred_val_log)

rmse_total = np.sqrt(mean_squared_error(y_val, y_pred))
rmse_per_target = np.sqrt(np.mean((y_val - y_pred)**2, axis=0))

print("Total RMSE:", rmse_total)
print("Per-target RMSE:", dict(zip(y_train.columns, rmse_per_target)))

Training target: TotalTimeStopped_p20


/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [19:29:00] WARNING: /workspace/src/context.cc:53: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [19:29:00] WARNING: /workspace/src/context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  self.starting_round = model.num_boosted_rounds()


[0]	train-rmse:0.99958	eval-rmse:1.00145
[50]	train-rmse:0.98212	eval-rmse:0.98611
[100]	train-rmse:0.96987	eval-rmse:0.97575
[150]	train-rmse:0.96043	eval-rmse:0.96820
[200]	train-rmse:0.95266	eval-rmse:0.96216
[250]	train-rmse:0.94612	eval-rmse:0.95721
[300]	train-rmse:0.93902	eval-rmse:0.95162
[350]	train-rmse:0.93172	eval-rmse:0.94586
[400]	train-rmse:0.92473	eval-rmse:0.94024
[450]	train-rmse:0.91920	eval-rmse:0.93587
[500]	train-rmse:0.91383	eval-rmse:0.93188
[550]	train-rmse:0.90903	eval-rmse:0.92834
[600]	train-rmse:0.90483	eval-rmse:0.92527
[650]	train-rmse:0.90088	eval-rmse:0.92239
[700]	train-rmse:0.89716	eval-rmse:0.91976
[750]	train-rmse:0.89331	eval-rmse:0.91705
[800]	train-rmse:0.88929	eval-rmse:0.91424
[850]	train-rmse:0.88588	eval-rmse:0.91200
[900]	train-rmse:0.88279	eval-rmse:0.91001
[950]	train-rmse:0.87958	eval-rmse:0.90793
[1000]	train-rmse:0.87680	eval-rmse:0.90620
[1050]	train-rmse:0.87374	eval-rmse:0.90421
[1100]	train-rmse:0.87063	eval-rmse:0.90228
[1150]	trai

/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [19:36:31] WARNING: /workspace/src/context.cc:53: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [19:36:31] WARNING: /workspace/src/context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  self.starting_round = model.num_boosted_rounds()


[0]	train-rmse:0.99926	eval-rmse:0.99953
[50]	train-rmse:0.97016	eval-rmse:0.97217
[100]	train-rmse:0.95184	eval-rmse:0.95531
[150]	train-rmse:0.93847	eval-rmse:0.94333
[200]	train-rmse:0.92862	eval-rmse:0.93477
[250]	train-rmse:0.92106	eval-rmse:0.92840
[300]	train-rmse:0.91371	eval-rmse:0.92214
[350]	train-rmse:0.90659	eval-rmse:0.91601
[400]	train-rmse:0.89893	eval-rmse:0.90917
[450]	train-rmse:0.89243	eval-rmse:0.90343
[500]	train-rmse:0.88679	eval-rmse:0.89858
[550]	train-rmse:0.88123	eval-rmse:0.89380
[600]	train-rmse:0.87581	eval-rmse:0.88914
[650]	train-rmse:0.87153	eval-rmse:0.88561
[700]	train-rmse:0.86678	eval-rmse:0.88167
[750]	train-rmse:0.86236	eval-rmse:0.87804
[800]	train-rmse:0.85777	eval-rmse:0.87423
[850]	train-rmse:0.85364	eval-rmse:0.87085
[900]	train-rmse:0.84962	eval-rmse:0.86758
[950]	train-rmse:0.84588	eval-rmse:0.86462
[1000]	train-rmse:0.84230	eval-rmse:0.86183
[1050]	train-rmse:0.83832	eval-rmse:0.85862
[1100]	train-rmse:0.83441	eval-rmse:0.85548
[1150]	trai

...

[0]	train-rmse:0.99958	eval-rmse:1.00499
[50]	train-rmse:0.98321	eval-rmse:0.99067
[100]	train-rmse:0.97156	eval-rmse:0.98080
[150]	train-rmse:0.96229	eval-rmse:0.97338
[200]	train-rmse:0.95501	eval-rmse:0.96780
[250]	train-rmse:0.94862	eval-rmse:0.96297
[300]	train-rmse:0.94185	eval-rmse:0.95772
[350]	train-rmse:0.93463	eval-rmse:0.95199
[400]	train-rmse:0.92821	eval-rmse:0.94694
[450]	train-rmse:0.92336	eval-rmse:0.94319
[500]	train-rmse:0.91855	eval-rmse:0.93960
[550]	train-rmse:0.91431	eval-rmse:0.93651
[600]	train-rmse:0.91029	eval-rmse:0.93365
[650]	train-rmse:0.90643	eval-rmse:0.93088
[700]	train-rmse:0.90272	eval-rmse:0.92825
[750]	train-rmse:0.89901	eval-rmse:0.92571
[800]	train-rmse:0.89516	eval-rmse:0.92301
[850]	train-rmse:0.89187	eval-rmse:0.92081
[900]	train-rmse:0.88885	eval-rmse:0.91887
[950]	train-rmse:0.88561	eval-rmse:0.91678
[1000]	train-rmse:0.88295	eval-rmse:0.91519
[1050]	train-rmse:0.87981	eval-rmse:0.91317
[1100]	train-rmse:0.87675	eval-rmse:0.91128
[1150]	trai

/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [20:02:37] WARNING: /workspace/src/context.cc:53: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [20:02:37] WARNING: /workspace/src/context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  self.starting_round = model.num_boosted_rounds()


[0]	train-rmse:0.99930	eval-rmse:1.00159
[50]	train-rmse:0.97404	eval-rmse:0.97802
[100]	train-rmse:0.95831	eval-rmse:0.96374
[150]	train-rmse:0.94651	eval-rmse:0.95329
[200]	train-rmse:0.93745	eval-rmse:0.94549
[250]	train-rmse:0.93020	eval-rmse:0.93938
[300]	train-rmse:0.92380	eval-rmse:0.93404
[350]	train-rmse:0.91718	eval-rmse:0.92838
[400]	train-rmse:0.91038	eval-rmse:0.92238
[450]	train-rmse:0.90420	eval-rmse:0.91695
[500]	train-rmse:0.89917	eval-rmse:0.91275
[550]	train-rmse:0.89353	eval-rmse:0.90784
[600]	train-rmse:0.88785	eval-rmse:0.90296
[650]	train-rmse:0.88347	eval-rmse:0.89926
[700]	train-rmse:0.87877	eval-rmse:0.89535
[750]	train-rmse:0.87409	eval-rmse:0.89147
[800]	train-rmse:0.87000	eval-rmse:0.88809
[850]	train-rmse:0.86557	eval-rmse:0.88441
[900]	train-rmse:0.86165	eval-rmse:0.88124
[950]	train-rmse:0.85772	eval-rmse:0.87806
[1000]	train-rmse:0.85465	eval-rmse:0.87571
[1050]	train-rmse:0.85074	eval-rmse:0.87258
[1100]	train-rmse:0.84704	eval-rmse:0.86965
[1150]	trai

/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [20:12:06] WARNING: /workspace/src/context.cc:53: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [20:12:06] WARNING: /workspace/src/context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  self.starting_round = model.num_boosted_rounds()


[0]	train-rmse:0.99916	eval-rmse:1.00065
[50]	train-rmse:0.97131	eval-rmse:0.97426
[100]	train-rmse:0.95468	eval-rmse:0.95893
[150]	train-rmse:0.94326	eval-rmse:0.94876
[200]	train-rmse:0.93487	eval-rmse:0.94151
[250]	train-rmse:0.92781	eval-rmse:0.93560
[300]	train-rmse:0.92097	eval-rmse:0.92977
[350]	train-rmse:0.91479	eval-rmse:0.92456
[400]	train-rmse:0.90786	eval-rmse:0.91842
[450]	train-rmse:0.90135	eval-rmse:0.91267
[500]	train-rmse:0.89659	eval-rmse:0.90860
[550]	train-rmse:0.89141	eval-rmse:0.90408
[600]	train-rmse:0.88610	eval-rmse:0.89943
[650]	train-rmse:0.88068	eval-rmse:0.89469
[700]	train-rmse:0.87554	eval-rmse:0.89024
[750]	train-rmse:0.87019	eval-rmse:0.88559
[800]	train-rmse:0.86516	eval-rmse:0.88116
[850]	train-rmse:0.86021	eval-rmse:0.87692
[900]	train-rmse:0.85580	eval-rmse:0.87322
[950]	train-rmse:0.85150	eval-rmse:0.86957
[1000]	train-rmse:0.84822	eval-rmse:0.86697
[1050]	train-rmse:0.84482	eval-rmse:0.86422
[1100]	train-rmse:0.84064	eval-rmse:0.86070
[1150]	trai

In [ ]:
# import xgboost as xgb
# from sklearn.metrics import mean_squared_error
# from sklearn.multioutput import MultiOutputRegressor

# print(x_train.shape)

# base_model = xgb.XGBRegressor(
#     objective='reg:squarederror', # regression
#     n_estimators=1000,
#     learning_rate=0.01,
#     max_depth=10,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     tree_method='hist',
#     eval_metric='rmse',
#     random_state=42,
#     device='cuda'
# )

# model = MultiOutputRegressor(base_model)

# model.fit(x_train, y_train)

# y_pred = model.predict(x_val)

# rmse = np.sqrt(mean_squared_error(y_val,y_pred))
# print('validation RMSE:',rmse)


In [ ]:
import numpy as np
print("y_train mean:", np.mean(y_train))
print("y_train std:", np.std(y_train))
print("y_train min:", np.min(y_train))
print("y_train max:", np.max(y_train))


Own Method-LSTM

In [ ]:
SEQ_LEN    = 12    # number of rows per sequence
BATCH_SIZE = 256   # trade-off between gradient stability and memory
EPOCHS     = 1     # TODO: increase later, 1 is only for short term testing
LR         = 1e-3
SEED       = 42


import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# fixed randomness
np.random.seed(SEED)
tf.random.set_seed(SEED)

# sort by time (Month, Hour)
# since LSTM need chronologically ordered
required_df_cols = ['Month', 'Hour', 'City', 'IntersectionId']
missing_any = False
for c in required_df_cols:
    if c not in df.columns:
        print("ERROR: Missing column in df:", c)
        missing_any = True

feature_cols = list(x_train.columns)   # your engineered X columns (one-hots, headings, cyclical time, etc.)
target_cols  = list(y_train.columns)   # the 6 regression targets

print("Number of feature columns:", len(feature_cols))
print("Feature columns:", feature_cols)
print("Target columns:", target_cols)

if missing_any:
    print("Please make sure df contains Month/Hour/City/IntersectionId before proceeding.")



def make_sequences_from_split(X_split, Y_split, seq_len):
    tmp = X_split.copy()

    # Rebuild a city key from your one-hot columns
    city_cols = [c for c in ['Atlanta','Boston','Chicago','Philadelphia'] if c in tmp.columns]
    if len(city_cols) == 0:
        raise ValueError("City one-hot columns not found in X. Expected any of: Atlanta/Boston/Chicago/Philadelphia.")
    tmp['CityKey'] = tmp[city_cols].idxmax(axis=1)

    # Reconstruct sortable cyclic angles for month/hour from your sin/cos features
    for need in ['hour_sin','hour_cos','month_sin','month_cos']:
        if need not in tmp.columns:
            raise ValueError(f"Missing column '{need}' in X needed to sort time for sequence building.")

    # atan2 → (-pi, pi]; map to [0, 2pi) so it's strictly sortable
    tmp['hour_angle']  = np.mod(np.arctan2(tmp['hour_sin'],  tmp['hour_cos']) + 2.0*np.pi, 2.0*np.pi)
    tmp['month_angle'] = np.mod(np.arctan2(tmp['month_sin'], tmp['month_cos']) + 2.0*np.pi, 2.0*np.pi)

    # Keep feature order consistent and valid
    valid_feat_cols = [c for c in feature_cols if c in tmp.columns]

    # Sort within (City, IntersectionId) by month then hour
    tmp_sorted = tmp.sort_values(['CityKey', 'IntersectionId', 'month_angle', 'hour_angle'])

    # Align Y to the same order
    Y_sorted = Y_split.loc[tmp_sorted.index]

    X_list, Y_list = [], []

    # Group by (CityKey, IntersectionId) so we never mix different locations in one sequence
    for (_, _), gdf in tmp_sorted.groupby(['CityKey','IntersectionId'], sort=False):
        if len(gdf) < seq_len:
            continue

        Xg = gdf[valid_feat_cols].to_numpy(dtype=np.float32)
        Yg = Y_sorted.loc[gdf.index].to_numpy(dtype=np.float32)

        # Sliding window; label is the window's last timestep
        for i in range(seq_len - 1, len(gdf)):
            start = i - seq_len + 1
            X_list.append(Xg[start:i+1])   # shape: (seq_len, feat_dim)
            Y_list.append(Yg[i])           # shape: (num_targets,)

    if not X_list:
        print("No sequences were created. Try reducing SEQ_LEN")
        return (
            np.empty((0, seq_len, len(valid_feat_cols)), np.float32),
            np.empty((0, Y_split.shape[1]), np.float32)
        )

    return np.stack(X_list).astype(np.float32), np.stack(Y_list).astype(np.float32)


# Build sequences for train/val (no scaling applied above)
Xtr_seq, ytr_seq = make_sequences_from_split(x_train, y_train, SEQ_LEN)
Xva_seq, yva_seq = make_sequences_from_split(x_val,   y_val,   SEQ_LEN)

print("X_train seq shape:", Xtr_seq.shape, "y_train shape:", ytr_seq.shape)
print("X_val   seq shape:", Xva_seq.shape, "y_val shape:", yva_seq.shape)

if Xtr_seq.shape[0] == 0 or Xva_seq.shape[0] == 0:
    print("No sequences built. Check SEQ_LEN or key columns in df (City/IntersectionId/Month/Hour).")


# LSTM head
def build_head_model(input_seq_len, input_feat_dim, out_dim=3, lstm_units=64, dense_units=128, dropout=0.1):
    model = keras.Sequential()
    model.add(layers.Input(shape=(input_seq_len, input_feat_dim)))
    model.add(layers.LSTM(lstm_units))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(dense_units, activation="relu"))
    model.add(layers.Dense(out_dim))  # outputs [p20, p50, p80]

    model.compile(
        optimizer=keras.optimizers.Adam(LR),
        loss="mse",
        metrics=[keras.metrics.RootMeanSquaredError(name="rmse")]
    )
    return model

# Prepare labels for TIME and DISTANCE heads (N,3)
time_cols = ['TotalTimeStopped_p20','TotalTimeStopped_p50','TotalTimeStopped_p80']
dist_cols = ['DistanceToFirstStop_p20','DistanceToFirstStop_p50','DistanceToFirstStop_p80']

time_idx = [target_cols.index(c) for c in time_cols]
dist_idx = [target_cols.index(c) for c in dist_cols]

ytr_time = ytr_seq[:, time_idx]
yva_time = yva_seq[:, time_idx]
ytr_dist = ytr_seq[:, dist_idx]
yva_dist = yva_seq[:, dist_idx]

# Callbacks
callbacks = [
    keras.callbacks.ReduceLROnPlateau(monitor="val_rmse", factor=0.5, patience=3, verbose=1),
    keras.callbacks.EarlyStopping(monitor="val_rmse", patience=7, restore_best_weights=True, verbose=1)
]

# 7) Train TIME model
print("\nTraining TIME model...")
time_model = build_head_model(Xtr_seq.shape[1], Xtr_seq.shape[2], out_dim=3)
time_model.fit(
    Xtr_seq, ytr_time,
    validation_data=(Xva_seq, yva_time),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

# Train DISTANCE model
print("\nTraining DISTANCE model...")
dist_model = build_head_model(Xtr_seq.shape[1], Xtr_seq.shape[2], out_dim=3)
dist_model.fit(
    Xtr_seq, ytr_dist,
    validation_data=(Xva_seq, yva_dist),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

# Validation prediction + per-column RMSE
print("\nPredicting on validation set...")

pred_time = time_model.predict(Xva_seq, batch_size=BATCH_SIZE, verbose=0)  # (N, 3)
pred_dist = dist_model.predict(Xva_seq, batch_size=BATCH_SIZE, verbose=0)  # (N, 3)

def rmse_each_column(y_true, y_pred, names):
    scores = {}
    for i, name in enumerate(names):
        mse  = mean_squared_error(y_true[:, i], y_pred[:, i])
        rmse = float(np.sqrt(mse))
        scores[name] = rmse
    return scores

rmse_time = rmse_each_column(yva_time, pred_time, time_cols)
rmse_dist = rmse_each_column(yva_dist, pred_dist, dist_cols)

print("\n=== RMSE===")
for k, v in rmse_time.items():
    print(k.ljust(30), ":", round(v, 4))
for k, v in rmse_dist.items():
    print(k.ljust(30), ":", round(v, 4))


Number of feature columns: 12
Feature columns: ['IntersectionId', 'EntryHeading', 'ExitHeading', 'Atlanta', 'Boston', 'Chicago', 'Philadelphia', 'hour_sin', 'hour_cos', 'Hour', 'Month', 'Weekend']
Target columns: ['TotalTimeStopped_p20', 'TotalTimeStopped_p50', 'TotalTimeStopped_p80', 'DistanceToFirstStop_p20', 'DistanceToFirstStop_p50', 'DistanceToFirstStop_p80']


ValueError: Missing column 'month_sin' in X needed to sort time for sequence building.